In [ ]:
import numpy as np
from scipy.io import mminfo,mmread
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.linalg import eig
from numpy import sqrt, dot, sum, abs, diag, array, pi
from scipy import sparse

from numpy.fft import rfft, rfftfreq

import sys
np.set_printoptions(threshold=sys.maxsize)
# np.set_printoptions(threshold=20)

Exercise 5 - Identification Techniques
===============================

This exercise is focused on identifying modal parameters from experiments.

Measurement data from a numerical impact hammer experiment on the plate supported by flexible springs is provided
in the file `measurement.txt`.
Time histories of acceleration at 24 points and of the force at the impact point were measured.
The point-coordinates are given in `measurement-coordinates.txt`.

You can load the data with
```python
from numpy import loadtxt
data = loadtxt('measurement.txt')
```

In [ ]:
from numpy import loadtxt
data = loadtxt('measurement.txt')

* Plot the time history for the force as well as its frequency spectrum.

In [ ]:
t = data[:,0] #Time vector
F = data[:,1] #Force vector

In [ ]:
plt.figure(figsize = [15,5])
plt.plot(t,F,label = 'F(t)')
plt.xlabel('t, in s')
plt.ylabel('F, in N')
plt.title('Applied Force')
plt.xlim([0,0.1])
plt.legend()
plt.grid()

In [ ]:
frequencies = rfftfreq(len(t),t[1] - t[0])
F_rfft = rfft(F)

* Plot the measured acceleration time histories and frequency spectra for the 4 corner points and the drive point (where the force is applied). Make sure to select appropriate time and frequency limits to only show useful data.

In [ ]:
plt.figure(figsize = [15,10])
plt.xlabel('t, in s')
plt.ylabel('a, in m/s^2')
plt.title('Acceleration time histories')
plt.xlim([0,1])

for i in range(2,26) :
    plt.plot(t,data[:,i],label = 'a' + str(i-1))
    
plt.legend(loc = 1)
plt.grid()

Now we calculate the frequeny spectra of the 4 corner points. Form the Numerical Experiment Jupyter Notebook we acquire following information:
 * **P01** - Left lower corner
 * **P06** - Right lower corner
 * **P19** - Left top corner
 * **P24** - Right top corner
 
For the injection point of the force: **P05**

In [ ]:
PO1_rfft = rfft(data[:,2])
PO6_rfft = rfft(data[:,7])
P19_rfft = rfft(data[:,20])
P24_rfft = rfft(data[:,25])

P05_rfft = rfft(data[:,6])

Now let's plot the frequency spectra

In [ ]:
plt.figure(figsize = [15,7])

plt.semilogy(frequencies,np.abs(PO1_rfft), label='PO1_rfft')
plt.semilogy(frequencies,np.abs(PO6_rfft), label='PO6_rfft')
plt.semilogy(frequencies,np.abs(P19_rfft), label='P19_rfft')
plt.semilogy(frequencies,np.abs(P24_rfft), label='P24_rfft')
plt.semilogy(frequencies,np.abs(P05_rfft), label='P05_rfft')

plt.ylabel('a [m/s^2]')
plt.xlabel('Frequency [Hz]')
plt.xlim(1,300)
plt.title('Frequency Spectra')
plt.legend()
plt.grid()

* Compute the accelerance and plot it for the drive point. 

Make sure to apply a suitable window to the data, to counteract the effect of poor signal to noise ratio in parts of the singal.
For decaying signal a decaring exponential window is suitable, e.g. use
```python
from scipy.signal import windows
frac = 1e-3 # choose something useful
W = windows.exponential(len(t), 0, -(len(t)-1) / np.log(frac), False)
```
Multiply the window to the data, before you do the FFT.


In [ ]:
from scipy.signal import windows
frac = 5e-3 # choose something useful
W = windows.exponential(len(t), 0, -(len(t)-1) / np.log(frac), False)

Let's first compare for one point how the window function influences the acceleration

In [ ]:
plt.figure(figsize = [15,5])
plt.plot(t,data[:,6]*W, label = "P01_window")
plt.plot(t,data[:,6], label = "P01")
plt.xlabel('t, in s')
plt.ylabel('P01, in m/sec^2')
plt.xlim([0,1])
plt.legend()
plt.grid()

In [ ]:
PO1_rfft_window = rfft(data[:,2]*W)
PO6_rfft_window = rfft(data[:,7]*W)
P19_rfft_window = rfft(data[:,20]*W)
P24_rfft_window = rfft(data[:,25]*W)

P05_rfft_window = rfft(data[:,6]*W)

In [ ]:
plt.figure(figsize = [15,7])

plt.semilogy(frequencies,np.abs(PO1_rfft_window), label='PO1_rfft_window')
plt.semilogy(frequencies,np.abs(PO6_rfft_window), label='PO6_rfft_window')
plt.semilogy(frequencies,np.abs(P19_rfft_window), label='P19_rfft_window')
plt.semilogy(frequencies,np.abs(P24_rfft_window), label='P24_rfft_window')
plt.semilogy(frequencies,np.abs(P05_rfft_window), label='P05_rfft_window')

plt.ylabel('a [m/s^2]')
plt.xlabel('Frequency [Hz]')
plt.xlim(1,300)
plt.title('Frequency Spectra')
plt.legend()
plt.grid()

* Compute the receptance from the accelerance and plot it for the drive point.

In [ ]:
P05_receptance = rfft(data[:,6]*W)/rfft(data[:,1]*W)

In [ ]:
plt.figure(figsize = [15,7])

plt.semilogy(frequencies,np.abs(P05_receptance))

plt.ylabel('a/F [m/s^2/N]')
plt.xlabel('Frequency [Hz]')
plt.xlim(1,300)
plt.title('Receptance from P05/F')
plt.grid()

## Task1: Find Peaks in the Transfer Functions
Use the Maximum Amplitude, Maximum Quadrature or Maximum Quadrature Component method to find all natural frequencies up to 150Hz.

For low frequency peaks it might me best to look at receptance curves, wheras for high frequenies the peaks in the accelerance might be more pronounced.
Some automatic peak finding algorithms are implemented in scipy. 
A useful and rather robust automatic peak-detection tool is the continuous wavelet transform, e.g. use
```python
from scipy.signal import find_peaks, find_peaks_cwt
```
Often the automatic selection has to be fine tuned by hand.

* Use an automatic tool to detect peaks in the drive-point accelerance, and plot the found peaks.
* Then adapt the result by hand (add shift peaks) until you have found all natural frequencies.
* Plot the magnitude, real and imaginary part of the fransfer functions around each peak in a suitable frequency range (~10Hz) to verify the peak location: all transfer functions in one plot, one plot per part, one plot per peak

In [ ]:
from scipy.signal import find_peaks, find_peaks_cwt

In [ ]:
# Use an automatic tool to detect peaks in the drive-point accelerance
peaks_idx_P05_rfft_window, _ = find_peaks(np.abs(P05_rfft_window), height=2000) 
# The height is set so that only realistic peaks are taken into account

In [ ]:
plt.figure(figsize = [15,7])
plt.ylabel('a [m/s^2]')
plt.xlabel('Frequency [Hz]')
plt.xlim(-1,151)
plt.title('Found peaks from P05 accelerance')
plt.grid()
plt.plot(frequencies, np.abs(P05_rfft_window))
plt.plot(frequencies[peaks_idx_P05_rfft_window], np.abs(P05_rfft_window)[peaks_idx_P05_rfft_window], 'x')
plt.show()

In [ ]:
# Plot the magnitude
bound = 10;

for n,i in enumerate(peaks_idx_P05_rfft_window):
    plt.figure(figsize = [15,7])

    lower_freq = frequencies[i]-bound
    upper_freq = frequencies[i]+bound
    
    lower_idx = (np.abs(frequencies - lower_freq)).argmin()
    upper_idx = (np.abs(frequencies - upper_freq)).argmin()
    
    plt.plot(frequencies[lower_idx:upper_idx]/frequencies[i], np.abs(P05_receptance[lower_idx:upper_idx]))
    plt.plot(frequencies[i]/frequencies[i], np.abs(P05_receptance[i]), 'x')
    
    plt.ylabel('magnitude')
    plt.xlabel('Frequency/Frequency_n [1]')
    plt.title('Point '+ str(n+1))
    plt.grid()

In [ ]:
# shift point 4 to the peak (Peak-Amplitude Methode)
peaks_idx_P05_rfft_window[3] = 253


plt.figure(figsize = [15,7])

lower_freq = frequencies[peaks_idx_P05_rfft_window[3]]-bound
upper_freq = frequencies[peaks_idx_P05_rfft_window[3]]+bound

lower_idx = (np.abs(frequencies - lower_freq)).argmin()
upper_idx = (np.abs(frequencies - upper_freq)).argmin()

plt.plot(frequencies[lower_idx:upper_idx]/frequencies[peaks_idx_P05_rfft_window[3]], np.abs(P05_receptance[lower_idx:upper_idx]))
plt.plot(frequencies[peaks_idx_P05_rfft_window[3]]/frequencies[peaks_idx_P05_rfft_window[3]], np.abs(P05_receptance[peaks_idx_P05_rfft_window[3]]), 'x')

plt.ylabel('magnitude')
plt.xlabel('Frequency/Frequency_n [1]')
plt.title('Point '+ str(4))
plt.grid()

In [ ]:
# Plot the real part
for n,i in enumerate(peaks_idx_P05_rfft_window):
    plt.figure(figsize = [15,7])

    lower_freq = frequencies[i]-bound
    upper_freq = frequencies[i]+bound
    
    lower_idx = (np.abs(frequencies - lower_freq)).argmin()
    upper_idx = (np.abs(frequencies - upper_freq)).argmin()
    
    plt.plot(frequencies[lower_idx:upper_idx]/frequencies[i], np.real(P05_receptance[lower_idx:upper_idx]))
    plt.plot(frequencies[i]/frequencies[i], np.real(P05_receptance[i]), 'x')
    
    plt.ylabel('real')
    plt.xlabel('Frequency/Frequency_n [1]')
    plt.title('Point '+ str(n+1))
    plt.grid()

In [ ]:
# Plot the imaginary part
for n,i in enumerate(peaks_idx_P05_rfft_window):
    plt.figure(figsize = [15,7])

    lower_freq = frequencies[i]-bound
    upper_freq = frequencies[i]+bound
    
    lower_idx = (np.abs(frequencies - lower_freq)).argmin()
    upper_idx = (np.abs(frequencies - upper_freq)).argmin()
    
    plt.plot(frequencies[lower_idx:upper_idx]/frequencies[i], np.imag(P05_receptance[lower_idx:upper_idx]))
    plt.plot(frequencies[i]/frequencies[i], np.imag(P05_receptance[i]), 'x')
    
    plt.ylabel('imaginary')
    plt.xlabel('Frequency/Frequency_n [1]')
    plt.title('Point '+ str(n+1))
    plt.grid()

In [ ]:
print('Peaks:', frequencies[peaks_idx_P05_rfft_window], 'Hz')

In [ ]:
# All TF in one plot
plt.figure(figsize = [15,7])
plt.ylabel('a/F [m/s^2/N]')
plt.xlabel('Frequency [Hz]')
plt.xlim(-1,151)
plt.title('All TF\'s in one plot')
plt.grid()

for ii in range(2, 26):
    
    Pi_rfft_window = rfft(data[:,ii]*W)
    
    peaks_idx_Pi_receptance, _ = find_peaks(np.abs(Pi_rfft_window), height=2000) 
    
    if ii == 6:
        plt.plot(frequencies, np.abs(Pi_rfft_window), linewidth=15)
        plt.plot(frequencies[peaks_idx_Pi_receptance], np.abs(Pi_rfft_window)[peaks_idx_Pi_receptance], 'x')
    else:
        plt.plot(frequencies, np.abs(Pi_rfft_window))
        plt.plot(frequencies[peaks_idx_Pi_receptance], np.abs(Pi_rfft_window)[peaks_idx_Pi_receptance], 'x')
        
plt.show()  

## Determine Damping Ratio
Determine the damping ratio of each natural frequency from the half power width.
Use the drive point receptance.

* plot the determined half power points on the transfer function for each peak

In [ ]:
half_power = np.abs(P05_receptance)/sqrt(2) #
print('Half power magnitude:', half_power[peaks_idx_P05_rfft_window])

In [ ]:
# find out to which index the half power points belong
index_half_power = np.zeros([len(peaks_idx_P05_rfft_window), 4],dtype = int)

for n,idx in enumerate(peaks_idx_P05_rfft_window):
    for i in range(idx, 0, -1):
        if half_power[idx] >= np.abs(P05_receptance)[i]:
            index_half_power[n,0] = i
            index_half_power[n,1] = i+1
            break

    for i in range(idx, len(frequencies)+1, 1):
        if half_power[idx] >= np.abs(P05_receptance)[i]:
            index_half_power[n,3] = i
            index_half_power[n,2] = i-1
            break

# linear interpolation between between the values found
freq_half_power = np.zeros([len(peaks_idx_P05_rfft_window),2])
    
for n,a in enumerate(index_half_power):
        freq_half_power[n,0] = ((frequencies[a[1]]-frequencies[a[0]])/
                                (np.abs(P05_receptance)[a[1]]-
                                 np.abs(P05_receptance)[a[0]]
                                ))*(half_power[peaks_idx_P05_rfft_window[n]]-np.abs(P05_receptance)[a[0]])+frequencies[a[0]]
        freq_half_power[n,1] = ((frequencies[a[3]]-frequencies[a[2]])/
                                (np.abs(P05_receptance)[a[3]]-
                                 np.abs(P05_receptance)[a[2]]
                                ))*(half_power[peaks_idx_P05_rfft_window[n]]-np.abs(P05_receptance)[a[2]])+frequencies[a[2]]

In [ ]:
# Plot the magnitude  
for n,i in enumerate(peaks_idx_P05_rfft_window):

    plt.figure(figsize = [15,7])

    lower_freq = frequencies[i]-bound
    upper_freq = frequencies[i]+bound
    
    lower_idx = (np.abs(frequencies - lower_freq)).argmin()
    upper_idx = (np.abs(frequencies - upper_freq)).argmin()
    
    if n == 4:
        P05_lower_idx = lower_idx
        P05_upper_idx = upper_idx
    else:
        pass
    
    plt.plot(frequencies[lower_idx:upper_idx]/frequencies[i], np.abs(P05_receptance[lower_idx:upper_idx]))
    plt.plot(frequencies[i]/frequencies[i], np.abs(P05_receptance[i]), 'x')
    plt.plot(freq_half_power[n,0]/frequencies[i], half_power[i], 'o')
    plt.plot(freq_half_power[n,1]/frequencies[i], half_power[i], 'o')
    
    plt.ylabel('magnitude')
    plt.xlabel('Frequency/Frequency_n [1]')
    plt.title('Point '+ str(n+1))
    plt.grid()

* what damping factors do you obtain?

In [ ]:
# Damping factor
Damping_factor = (freq_half_power[:,1]-freq_half_power[:,0])/(2*frequencies[peaks_idx_P05_rfft_window])

print('Damping Factors: ', Damping_factor)

In [ ]:
# shift first half power of point 5 into valley
valley_idx = np.argmin(np.abs(P05_receptance[peaks_idx_P05_rfft_window[-2]:peaks_idx_P05_rfft_window[-1]]))
freq_half_power[4,0] = frequencies[peaks_idx_P05_rfft_window[-2]:peaks_idx_P05_rfft_window[-1]][valley_idx]

lower_freq = frequencies[peaks_idx_P05_rfft_window[-1]]-bound
upper_freq = frequencies[peaks_idx_P05_rfft_window[-1]]+bound

lower_idx = (np.abs(frequencies - lower_freq)).argmin()
upper_idx = (np.abs(frequencies - upper_freq)).argmin()

plt.figure(figsize = [15,7])
plt.plot(frequencies[lower_idx:upper_idx]/frequencies[peaks_idx_P05_rfft_window[-1]],
         np.abs(P05_receptance[lower_idx:upper_idx]))
plt.plot(frequencies[peaks_idx_P05_rfft_window[-1]]/frequencies[peaks_idx_P05_rfft_window[-1]],
         np.abs(P05_receptance[peaks_idx_P05_rfft_window[-1]]), 'x')
plt.plot(freq_half_power[4,0]/frequencies[peaks_idx_P05_rfft_window[-1]], half_power[peaks_idx_P05_rfft_window[-1]], 'o')
plt.plot(freq_half_power[4,1]/frequencies[peaks_idx_P05_rfft_window[-1]], half_power[peaks_idx_P05_rfft_window[-1]], 'o')

plt.ylabel('magnitude')
plt.xlabel('Frequency/Frequency_n [1]')
plt.title('Point '+ str(5))
plt.grid()

In [ ]:
# Damping factors after shifting point 5
Damping_factor = (freq_half_power[:,1]-freq_half_power[:,0])/(2*frequencies[peaks_idx_P05_rfft_window])

print('Damping Factors: ', Damping_factor)

## Circle Fitting
Employ the circle fitting algorithm to determine natural frequencies and damping factors.

* Develop a function that fits a circle in the nyquist plane to given data points (receptance values and corresponding frequencies). It should return the natural frequency, damping factor and complex valued modal constant.

In [ ]:
def CircleFitting(receptance,f,printFlag = 0) :
    x = np.real(receptance)
    y = np.imag(receptance)
    w = 2*np.pi*f
    dw = w[1] - w[0]
    
    ## DO CIRCLE FITTING STUFF TO THE DATA
    L = len(x)
    A = np.zeros((3,3))
    
    A[0,0] = np.sum(x*x)
    A[1,1] = np.sum(y*y)
    A[2,2] = L
    
    A[0,1] = A[1,0] = np.sum(x*y)
    A[0,2] = A[2,0] = -np.sum(x)
    A[1,2] = A[2,1] = -np.sum(y)
    
    B = np.zeros((3,1))
    B[0] = -np.sum(x*x*x)-np.sum(x*y*y)
    B[1] = -np.sum(y*y*y)-np.sum(x*x*y)
    B[2] = np.sum(x*x)+np.sum(y*y)
    
    sol = np.linalg.solve(A,B)
    
    ## CIRCLE DATA
    xk = -1/2*sol[0]
    yk = -1/2*sol[1]
    Rk = np.sqrt(sol[2] + xk**2 + yk**2)
    
    # Generate spokes for plotting
    pointers_x = x - xk
    pointers_y = y - yk
    
    frequency_spacing = np.zeros((len(pointers_x)-1))
    
    ## CALCULATE ANGLES BETWEEN SPOKES
    for i in range(0,len(pointers_x)-1):
        pointer_1 = np.array((pointers_x[i],pointers_y[i]))
        pointer_2 = np.array((pointers_x[i+1],pointers_y[i+1]))
        frequency_spacing[i] = np.arccos(np.dot(pointer_1,pointer_2)/
                                         np.linalg.norm(pointer_1)/
                                         np.linalg.norm(pointer_2))
    
    ## CALCULATE GRADIENT
    frequency_spacing = np.cumsum(frequency_spacing)
    dg_dw = np.gradient(frequency_spacing,dw)
    d2g_dw2 = np.gradient(dg_dw,dw)
    idx_max = np.argmax(dg_dw)
    
    # Find Maximum and Minium Values of the function --> somewhere between them we have the 0 crossing
    max_idx_d2g_dw2 = (np.argmax(d2g_dw2))
    min_idx_d2g_dw2 = (np.argmin(d2g_dw2))
    
    # Flip values of d2g_dw2, need the function to be increasing ---> form - values to + values
    x_values = np.flip(d2g_dw2[max_idx_d2g_dw2:min_idx_d2g_dw2+1])
    y_values = np.flip(f[max_idx_d2g_dw2:min_idx_d2g_dw2+1])
    
    # Find zero values
    freq_zero_crossing = np.interp(0.0,x_values,y_values)    
    
    # Find x,y values of wk on given data
    x_wk = np.interp(freq_zero_crossing,f,x)
    y_wk = np.interp(freq_zero_crossing,f,y)
    
    ## FIND wk
    wk = freq_zero_crossing*2*np.pi
    idx_a_b = np.abs(w-wk).argsort()[:2]
    wa = w[idx_a_b[0]]
    wb = w[idx_a_b[1]]
    
    # Now build the vector to wk and find right point of w_k
    # We seek the intersection point of the vector from center to wk with the circle 
    k = (y_wk - yk)/(x_wk - xk) # Gradient of the pointer_wk
    x_circ = np.array([Rk/np.sqrt(1+k**2), -Rk/np.sqrt(1+k**2)]) # x on circle, relative to circle center, quadratic sol.
    y_circ = k*x_circ # y on circle, relative to circle center
    x_circ = x_circ + xk # now in absolute reference system
    y_circ = y_circ + yk # now in absolute reference system
    
    # Lets find which one is wk and which one is Dk
    idx_wk = np.abs(x_circ - x_wk).argmin()
    idx_Dk = np.abs(x_circ - x_wk).argmax()
    x_wk_circ = float(x_circ[idx_wk])
    y_wk_circ = float(y_circ[idx_wk])
    x_Dk_circ = float(x_circ[idx_Dk])
    y_Dk_circ = float(y_circ[idx_Dk])
    
    # FIND Dk
    Dk = np.complex(x_Dk_circ,y_Dk_circ)

    # make some pointer/vector magic
    pointer_wk_DK = np.array([[x_wk_circ,x_Dk_circ],[y_wk_circ,y_Dk_circ]])
    pointer_wa_DK = np.array([[x[idx_a_b[0]],x_Dk_circ],[y[idx_a_b[0]],y_Dk_circ]])
    pointer_wb_DK = np.array([[x[idx_a_b[1]],x_Dk_circ],[y[idx_a_b[1]],y_Dk_circ]])
    pointer_Dk_down = np.array([[x_Dk_circ,x_Dk_circ],[y_wk_circ,y_Dk_circ]])
    Dk_coord = np.array([x_Dk_circ,y_Dk_circ])

    # Calculate Phia Phib Phik
    # CALCULATE Phi_a / ...
    Phi_a = np.arccos(np.dot(pointer_wk_DK[:,0] - Dk_coord,pointer_wa_DK[:,0] - Dk_coord)/
                      np.linalg.norm(pointer_wk_DK[:,0] - Dk_coord)/np.linalg.norm(pointer_wa_DK[:,0] - Dk_coord))
    Phi_b = np.arccos(np.dot(pointer_wk_DK[:,0] - Dk_coord,pointer_wb_DK[:,0] - Dk_coord)/
                      np.linalg.norm(pointer_wk_DK[:,0] - Dk_coord)/np.linalg.norm(pointer_wb_DK[:,0] - Dk_coord))
    Phi_k = np.arccos(np.dot(pointer_wk_DK[:,0] - Dk_coord,pointer_Dk_down[:,0] - Dk_coord)/
                      np.linalg.norm(pointer_wk_DK[:,0] - Dk_coord)/np.linalg.norm(pointer_Dk_down[:,0] - Dk_coord))

    zeta_k = (wa**2 - wb**2)/2/wk**2/(np.tan(Phi_a) + np.tan(Phi_b))
    C_k = 4*Rk*wk**2*zeta_k*np.exp(1j*Phi_k)
    
    H_w = C_k/(wk**2 - w*w + 2*1j*wk*zeta_k*w) + Dk

    if printFlag == 1 :
        
        # Calc circle data
        theta = np.linspace(0,2*np.pi,100)
        x_circle = Rk*np.cos(theta) + xk
        y_circle = Rk*np.sin(theta) + yk

        plt.figure(figsize = [15,7])
        
        # Plot circle data and measurement data
        plt.plot(x,y,label = 'data')
        plt.plot(x_circle,y_circle, '--', label = 'circle fit')
        
        # pointers to wa wb wk from Dk / vector Dk down
        plt.plot(pointer_wk_DK[0],pointer_wk_DK[1],'r--',linewidth = 3)
        plt.plot(pointer_wa_DK[0],pointer_wa_DK[1],'m-',linewidth = 2)
        plt.plot(pointer_wb_DK[0],pointer_wb_DK[1],'m-',linewidth = 2)
        plt.plot(pointer_Dk_down[0],pointer_Dk_down[1],'b-',linewidth = 2)

        # Plot the center of the circle
        plt.plot(xk,yk, 'o',label = 'center', markersize=10)
        plt.text(xk,yk,'Mk',fontsize=15)
        
        # Plot w_k on the given data
        plt.plot(x_wk,y_wk, 'x',label = 'w_k', markersize=10)
        
        # plot w_k on circle
        plt.plot(x_wk_circ,y_wk_circ,'o', markersize=10, label = 'w_k_circ')
        plt.text(x_wk_circ,y_wk_circ,'w_k',fontsize=15)
        plt.plot(x_Dk_circ,y_Dk_circ,'o', markersize=10, label = 'D_K')
        plt.text(x_Dk_circ,y_Dk_circ,'D_k',fontsize=15)
        
        # Plot wa and wb
        plt.plot(x[idx_a_b[0]],y[idx_a_b[0]],'o',label = 'w_a', markersize=10)
        plt.text(x[idx_a_b[0]],y[idx_a_b[0]],'wa',fontsize=15)
        plt.plot(x[idx_a_b[1]],y[idx_a_b[1]],'o',label = 'w_b', markersize=10)
        plt.text(x[idx_a_b[1]],y[idx_a_b[1]],'wb',fontsize=15)

        # Plot spokes for all
        #         for i,idx in enumerate(pointers_x) :
        #             plt.plot([xk,x[i]],[yk,y[i]],'--')

        plt.title('Circle Fitting, fk = ' + str(wk/2/np.pi))
        plt.ylabel('imag(H(w))')
        plt.xlabel('real(H(w))')
        plt.legend()
        plt.axis('equal')
        plt.grid()   

        # Plot cumsum of gamma, plot first and second derivative and some fancy stuff
        # ----------------------------------- DEBUGGING STUFF --------------------------------------------------
        plt.figure(figsize = [15,10])
        plt.subplot(2,2,1)
        plt.plot(f,np.abs(receptance), label = 'measured recepetance')
        plt.plot(f,np.abs(H_w),label = 'circle fit')
        plt.ylabel('abs(H(w))')
        plt.title('Receptance')
        plt.grid()
        plt.legend()

        plt.subplot(2,2,2)
        plt.plot(f[0:-1],frequency_spacing)
        plt.ylabel('gamma [rad]')
        plt.title('Gamma')
        plt.grid()
        
        plt.subplot(2,2,3)
        plt.plot(f[0:-1],dg_dw)
        plt.title('dg/dw')
        plt.ylabel('d2g/dw2 [rad*sec]')
        plt.xlabel('frequencies [Hz]')
        plt.grid()

        plt.subplot(2,2,4)
        plt.plot(f[0:-1],d2g_dw2)
        plt.plot(f[int(idx_a_b[0])],d2g_dw2[idx_a_b[0]],'*',label = 'w_a')
        plt.plot(f[int(idx_a_b[1])],d2g_dw2[idx_a_b[1]],'*',label = 'w_b')
        plt.plot(freq_zero_crossing,0,'o',label = 'w_k')
        plt.title('d2g/dw2')
        plt.ylabel('d2g/dw2 [rad*sec^2]')
        plt.xlabel('frequencies [Hz]')
        plt.grid()
        plt.legend()
        plt.show()
    
    return wk, zeta_k, C_k[0], Dk, H_w

* First test your function on a nice, prominant peak of the drive point receptance (e.g. around 73Hz): Select suitable data points (frequency range) for the fit. Plot your fit in the Nyquist plane. Also plot the transfer function estimation by the cicle fit, i.e. $$ H(\omega) = \frac{C_k}{\omega_k^2 - \omega^2 + 2j \omega_k\zeta_k\omega}+D_k$$ in a wider frequency range, and compare it to the data to see how the fit performs.

* Use your function and determine suitable frequency ranges to fit all natural frequencies. For some it might be useful to switch to a different transfer function, where the peak under consideration is more prominant.

#### The plotting of HW on a wider range is done in the last point below.

Range for Circle Fitting:

- Point 1
    - [0,15] -> 1,2,7,8,23,24 measurement point
    - [0,31] -> rest

- Point 2
    - [32,70] -> 3,9,10,15,21,22
    - [32,150] -> rest

- Point 3
    - [151,230] -> rest !Caution! Point 24

- Point 4
    - [240,262] -> 2,3,6,7,10,11,14,15,18,19,22
    - [231,262] -> rest
    - [240,255] -> 23

- Point 5
    - [262,280] -> 3,22
    - [262,350] -> rest
    
The one generated for only P05 made problems with with some points, just for reference
```py
frequency_indexes = np.array([[0,31],
                             [32,150],
                             [151,230],
                             [231,262],
                             [262,350]
                             ])
```

In [ ]:
# # ------------------------------ USED TO DEBUG AND FIND RANGE FOR CIRCLE FITTING ------------------------------------
# window = range(240,255)
# plt.figure(figsize = [15,50])
    
# for i in range(2,26):
#     Pi_receptance = rfft(data[:,i]*W)/rfft(data[:,1]*W)
    
#     plt.subplot(13,2,i-1)
#     plt.title('Plot for measurementpoint: ' + str(i-1))
#     plt.grid()
#     plt.plot(np.abs(Pi_receptance[window]))
# plt.show()

In [ ]:
# ------------------------------ USED TO DEBUG AND FIND RANGE FOR CIRCLE FITTING ------------------------------------
# window = range(262,280)
# plt.figure(figsize = [15,7])

# i = 3
# Pi_receptance = rfft(data[:,i + 1]*W)/rfft(data[:,1]*W)
    
# plt.title('Plot for measurementpoint: ' + str(i))
# plt.grid()
# plt.plot(np.abs(Pi_receptance[window]))
# plt.show()

In [ ]:
## EXAMPLE OF THE CIRCLE FITTING TOOL
lower_idx = 32
upper_idx = 150

f = frequencies[lower_idx:upper_idx]
receptance = P05_receptance[lower_idx:upper_idx]

# Toggle plots ON/Off with 0 or 1 as flag in Circle Fitting
[wk, zeta_k, C_k, D_k, H_w] = CircleFitting(receptance,f,1)

print("------------------------- Peak -------------------------")
print("Natural Frequency: ", wk/2/np.pi)
print("Damping Factor: ", zeta_k)
print("Modal Constant: ", C_k)
print()

* Use the circle fit algorithm to determine the modal constants for all measured locations (=recpetance functions). Use the frequency ranges (around the peaks) determined above.

In [ ]:
# Build some containers to save all the fittings
size_container = (len(peaks_idx_P05_rfft_window),24)

wk_all = np.zeros(size_container, dtype = float)
zeta_k_all = np.zeros(size_container, dtype = float)
C_k_all = np.zeros(size_container, dtype = complex)
D_k_all = np.zeros(size_container, dtype = complex)

In [ ]:
# Lets do it for the first point
# ------------------------------
#     - [0,15] -> 1,2,7,8,23,24
#     - [0,31] -> rest
indexi = np.array([1,2,7,8,23,24])
k = 0

for i in range(1,25):
    if i in indexi:
        fRange = range(0,15)
    else:
        fRange = range(0,31)
        
    Pi_receptance = rfft(data[:,i+1]*W)/rfft(data[:,1]*W)
    f = frequencies[fRange]
    receptance = Pi_receptance[fRange]
    
    # Toggle plots ON/Off with 0 or 1 as flag in Circle Fitting
    [wk, zeta_k, C_k, D_k, H_w] = CircleFitting(receptance,f,0)
    
    wk_all[k,i-1] = wk
    zeta_k_all[k,i-1] = zeta_k
    C_k_all[k,i-1] = C_k
    D_k_all[k,i-1] = D_k
    
#     print("------------------------- Point " + str(i) + "-------------------------")
#     print("Natural Frequency: ", wk/2/np.pi)
#     print("Damping Factor: ", zeta_k)
#     print("Modal Constant: ", C_k)
#     print()

In [ ]:
# Lets do it for the Second point
# ------------------------------
#     - [31,70] -> 3,9,10,15,21,22
#     - [31,150] -> rest
indexi = np.array([3,9,10,15,21,22])
k = 1

for i in range(1,25):
    if i in indexi:
        fRange = range(31,70)
    else:
        fRange = range(31,150)
        
    Pi_receptance = rfft(data[:,i+1]*W)/rfft(data[:,1]*W)
    f = frequencies[fRange]
    receptance = Pi_receptance[fRange]
    
    # Toggle plots ON/Off with 0 or 1 as flag in Circle Fitting
    [wk, zeta_k, C_k, D_k, H_w] = CircleFitting(receptance,f,0)
    
    wk_all[k,i-1] = wk
    zeta_k_all[k,i-1] = zeta_k
    C_k_all[k,i-1] = C_k
    D_k_all[k,i-1] = D_k
    
#     print("------------------------- Point " + str(i) + "-------------------------")
#     print("Natural Frequency: ", wk/2/np.pi)
#     print("Damping Factor: ", zeta_k)
#     print("Modal Constant: ", C_k)
#     print()

In [ ]:
# Lets do it for the Third point
# ------------------------------
# [151,230] -> all
fRange = range(151,230)
k = 2

for i in range(1,25):         
    Pi_receptance = rfft(data[:,i+1]*W)/rfft(data[:,1]*W)
    f = frequencies[fRange]
    receptance = Pi_receptance[fRange]
    
    # Toggle plots ON/Off with 0 or 1 as flag in Circle Fitting
    [wk, zeta_k, C_k, D_k, H_w] = CircleFitting(receptance,f,0)
    
    wk_all[k,i-1] = wk
    zeta_k_all[k,i-1] = zeta_k
    C_k_all[k,i-1] = C_k
    D_k_all[k,i-1] = D_k
    
#     print("------------------------- Point " + str(i) + "-------------------------")
#     print("Natural Frequency: ", wk/2/np.pi)
#     print("Damping Factor: ", zeta_k)
#     print("Modal Constant: ", C_k)
#     print()

In [ ]:
# Lets do it for the Fourth point
# ------------------------------
# [240,262] -> 2,3,6,7,10,11,14,15,18,19,22
# [231,262] -> rest
# [240,255] -> 23
indexi = np.array([2,3,6,7,10,11,14,15,18,19,22])
indexii = np.array([23])
k = 3

for i in range(1,25):
    if i in indexi:
        fRange = range(240,262)
    elif i in indexii:
        fRange = range(240,255)
    else:
        fRange = range(231,262)
    
    Pi_receptance = rfft(data[:,i+1]*W)/rfft(data[:,1]*W)
    f = frequencies[fRange]
    receptance = Pi_receptance[fRange]
    
    # Toggle plots ON/Off with 0 or 1 as flag in Circle Fitting
    [wk, zeta_k, C_k, D_k, H_w] = CircleFitting(receptance,f,0)
    
    wk_all[k,i-1] = wk
    zeta_k_all[k,i-1] = zeta_k
    C_k_all[k,i-1] = C_k
    D_k_all[k,i-1] = D_k
    
#     print("------------------------- Point " + str(i) + "-------------------------")
#     print("Natural Frequency: ", wk/2/np.pi)
#     print("Damping Factor: ", zeta_k)
#     print("Modal Constant: ", C_k)
#     print()

In [ ]:
# Lets do it for the fifth point
# ------------------------------
#     - [262,280] -> 3,22
#     - [262,350] -> rest
indexi = np.array([3,22])
k = 4

for i in range(1,25):
    if i in indexi:
        fRange = range(262,280)
    else:
        fRange = range(262,350)

    Pi_receptance = rfft(data[:,i+1]*W)/rfft(data[:,1]*W)
    f = frequencies[fRange]
    receptance = Pi_receptance[fRange]
    
    # Toggle plots ON/Off with 0 or 1 as flag in Circle Fitting
    [wk, zeta_k, C_k, D_k, H_w] = CircleFitting(receptance,f,0)
    
    wk_all[k,i-1] = wk
    zeta_k_all[k,i-1] = zeta_k
    C_k_all[k,i-1] = C_k
    D_k_all[k,i-1] = D_k
    
#     print("------------------------- Point " + str(i) + "-------------------------")
#     print("Natural Frequency: ", wk/2/np.pi)
#     print("Damping Factor: ", zeta_k)
#     print("Modal Constant: ", C_k)
#     print()

* Do the natural frequecies and damping factors obtained from different transfer functions differ?

In [ ]:
# Plot wks
index = np.arange(1,25)

plt.figure(figsize = [15,17])

plt.subplot(3,2,1)
plt.bar(index, wk_all[0,:])
plt.xlabel('measurement point')
plt.ylabel('freuqency [rad/s]')
plt.title('Natural Frequency: 1')
plt.yscale('log')

plt.subplot(3,2,2)
plt.bar(index, wk_all[1,:])
plt.xlabel('measurement point')
plt.ylabel('freuqency [rad/s]')
plt.title('Natural Frequency: 2')
plt.yscale('log')

plt.subplot(3,2,3)
plt.bar(index, wk_all[2,:])
plt.xlabel('measurement point')
plt.ylabel('freuqency [rad/s]')
plt.title('Natural Frequency: 3')
plt.yscale('log')

plt.subplot(3,2,4)
plt.bar(index, wk_all[3,:])
plt.xlabel('measurement point')
plt.ylabel('freuqency [rad/s]')
plt.title('Natural Frequency: 4')
plt.yscale('log')

plt.subplot(3,2,5)
plt.bar(index, wk_all[4,:])
plt.xlabel('measurement point')
plt.ylabel('freuqency [rad/s]')
plt.title('Natural Frequency: 5')
plt.yscale('log')

plt.show()

In [ ]:
# Plot damping factors
index = np.arange(1,25)

plt.figure(figsize = [15,17])

plt.subplot(3,2,1)
plt.bar(index, zeta_k_all[0,:])
plt.xlabel('measurement point')
plt.ylabel('damping factor')
plt.title('Natural Frequency: 1')
# plt.yscale('log')

plt.subplot(3,2,2)
plt.bar(index, zeta_k_all[1,:])
plt.xlabel('measurement point')
plt.ylabel('damping factor')
plt.title('Natural Frequency: 2')
# plt.yscale('log')

plt.subplot(3,2,3)
plt.bar(index, zeta_k_all[2,:])
plt.xlabel('measurement point')
plt.ylabel('damping factor')
plt.title('Natural Frequency: 3')
# plt.yscale('log')

plt.subplot(3,2,4)
plt.bar(index, zeta_k_all[3,:])
plt.xlabel('measurement point')
plt.ylabel('damping factor')
plt.title('Natural Frequency: 4')
# plt.yscale('log')

plt.subplot(3,2,5)
plt.bar(index, zeta_k_all[4,:])
plt.xlabel('measurement point')
plt.ylabel('damping factor')
plt.title('Natural Frequency: 5')
# plt.yscale('log')

plt.show()

* Use the moda parameters obtained via the circle fit to estimate the drive point recepance. Plot the measured repeptance, the single dregree of freedom estimates from the circle fit, as well as their sum (all in one plot to compare).

In [ ]:
# Drive-Point --> P05 --> index 4
max_idx = np.abs(frequencies-100).argmin();
w = frequencies[0:max_idx]*2*np.pi
H_w_sum = np.zeros_like(frequencies[0:max_idx], dtype = 'complex')

plt.figure(figsize = [15,7])
for i, wk in enumerate(wk_all[:,4]):
    H_w = C_k_all[i,4]/(wk**2 - w*w + 2*1j*wk*zeta_k_all[i,4]*w) +  D_k_all[i,4]
    H_w_sum = H_w_sum + H_w
 
    plt.plot(frequencies[0:max_idx],np.abs(H_w),label = 'Natural Frequency ' + str(i))
plt.xlim([0,100])
plt.ylabel('abs(H(w))')
plt.grid()
plt.legend()
plt.show()

# Plot Measurement and sum of ours
# shift = 22
shift = 0
plt.figure(figsize = [15,7])
plt.plot(frequencies[0:max_idx],np.abs(H_w_sum)-shift, label = 'circle fitted receptance')
plt.plot(frequencies[0:max_idx],np.abs(P05_receptance[0:max_idx]), label = 'measured receptance')
plt.xlim([0,100])
plt.grid()
plt.ylabel('abs(H(w))')
plt.xlabel('frequency [Hz]')
plt.legend()
plt.show()

## Task 2: Compute the Mode Shapes

Having obtained natural frequencies and damping factors, from peak-picking (maximum maplitude, quadrature component, ...) use the recepance curves to determine the modal constants and mode shapes.
The mode shapes will be complex valued in ganeral (due to the damping present in the system). As the are complex-valued you can display them as real/imaginary part, or absolute value and phase. Try both!

In [ ]:
def plotmode2d(v,x,y,lim=None) :
    """plots a mode v defined at the points (x,y)"""
    if lim==None:
        lim = np.max(np.abs(v))
    plt.tricontourf(x,y,v,cmap=plt.get_cmap('RdBu_r'),vmin=-lim,vmax=lim)
#     plt.set_aspect('equal')
    plt.xticks([])
    plt.yticks([])
    #fig.tight_layout()

* Compute and plot the mode shapes from the peak-picking procedure

In [ ]:
coordinates = loadtxt('measurement-coordinates.txt')

Mode_shapes_peakpicking = np.zeros((24,5),dtype= complex)

for k in range(0,5):
    H_ai = np.zeros(24, dtype = 'complex')
    zeta_k = Damping_factor[k]
    wk = 2*np.pi*frequencies[peaks_idx_P05_rfft_window[k]]

    for i in range(0,24):
        a = rfft(data[:,i+2]*W)/rfft(data[:,1]*W)
        H_ai[i] = a[peaks_idx_P05_rfft_window[k]]

    C = H_ai*2*1j*wk**2

    vak = np.sqrt(2*1j*zeta_k*wk**2*P05_receptance[peaks_idx_P05_rfft_window[k]])
    
    vbk = C/vak
    Mode_shapes_peakpicking[:,k]= vbk
    
    plt.figure(figsize=[15,7])
    plt.subplot(2,2,1)
    plotmode2d(np.abs(vbk),coordinates[:,1],coordinates[:,2])
    plt.title('Mode %i @ %f Hz'%(k+1,frequencies[peaks_idx_P05_rfft_window[k]]))
    
    plt.subplot(2,2,2)
    plotmode2d(np.angle(vbk),coordinates[:,1],coordinates[:,2])
    plt.title('Mode %i Phase'%(k+1))
    
    plt.subplot(2,2,3)
    plotmode2d(np.real(vbk),coordinates[:,1],coordinates[:,2])
    plt.title('Mode %i Real'%(k+1))
    
    plt.subplot(2,2,4)
    plotmode2d(np.imag(vbk),coordinates[:,1],coordinates[:,2])
    plt.title('Mode %i Imaginary'%(k+1))

* Compute and plot the modes from circle fitting

In [ ]:
Mode_shapes_circle = np.zeros((24,5),dtype = complex)

for i in range(0,5):
    vbk_circle = C_k_all[i,:]/np.sqrt(C_k_all[i,4])
    Mode_shapes_circle[:,i]= vbk_circle
    
    plt.figure(figsize=[15,7])
    plt.subplot(2,2,1)
    plotmode2d(np.abs(vbk_circle),coordinates[:,1],coordinates[:,2])
    plt.title('Mode %i @ %f Hz'%(i+1,np.mean(wk_all[i,:]/2/pi)))
    
    plt.subplot(2,2,2)
    plotmode2d(np.angle(vbk_circle),coordinates[:,1],coordinates[:,2])
    plt.title('Mode %i Phase'%(i+1))
    
    plt.subplot(2,2,3)
    plotmode2d(np.real(vbk_circle),coordinates[:,1],coordinates[:,2])
    plt.title('Mode %i Real'%(i+1))
    
    plt.subplot(2,2,4)
    plotmode2d(np.imag(vbk_circle),coordinates[:,1],coordinates[:,2])
    plt.title('Mode %i Imaginary'%(i+1))

* animate selected mode shapes. What do you observe?

In [ ]:
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

def giveMeAnimation(v,coordinates,whatTOdo,modenumber):
    plt.rcParams["animation.html"] = "jshtml"
    
    x = coordinates[:,1]
    y = coordinates[:,2]
    z = v

    x = np.reshape(x,(4,6))
    y = np.reshape(y,(4,6))
    z = np.reshape(z,(4,6))

    #------------------------------------------------------------
    # set up figure and animation
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    sc = ax.plot_surface(x,y,z)
    
    def animate(i):
        # scale factor for plotting
        s = np.sin(1/25*2*np.pi*i)
        Vs = z*s # sinusoidal scaled modeshape vector
        
        ax.clear()
        sc = ax.plot_surface(x,y,Vs,cmap=cm.brg,linewidth=0, antialiased=False)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        ax.set_zlim([-z.max(),z.max()])
        ax.set_title("Mode: %i" %(modenumber))

        return sc

    ani = animation.FuncAnimation(fig, animate, frames=50,
                                  interval=100)

    # # save the animation as an mp4.  This requires ffmpeg or mencoder to be
    # # installed.  The extra_args ensure that the x264 codec is used, so that
    # # the video can be embedded in html5.  You may need to adjust this for
    # # your system: for more information, see
    # # http://matplotlib.sourceforge.net/api/animation_api.html
    
    if whatTOdo == 'Save' :
        ani.save('modalanalyse_mode_' + str(i) + '.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
    elif whatTOdo == 'justShow':
        plt.close()
        return ani

In [ ]:
giveMeAnimation(np.abs(Mode_shapes_peakpicking[:,0]),coordinates,'justShow',1)

In [ ]:
giveMeAnimation(np.abs(Mode_shapes_peakpicking[:,1]),coordinates,'justShow',2)

In [ ]:
giveMeAnimation(np.abs(Mode_shapes_peakpicking[:,2]),coordinates,'justShow',3)

In [ ]:
giveMeAnimation(np.abs(Mode_shapes_peakpicking[:,3]),coordinates,'justShow',4)

In [ ]:
giveMeAnimation(np.abs(Mode_shapes_peakpicking[:,4]),coordinates,'justShow',5)

In [ ]:
giveMeAnimation(np.abs(Mode_shapes_circle[:,0]),coordinates,'justShow',1)

In [ ]:
giveMeAnimation(np.abs(Mode_shapes_circle[:,1]),coordinates,'justShow',2)

In [ ]:
giveMeAnimation(np.abs(Mode_shapes_circle[:,2]),coordinates,'justShow',3)

In [ ]:
giveMeAnimation(np.abs(Mode_shapes_circle[:,3]),coordinates,'justShow',4)

In [ ]:
giveMeAnimation(np.abs(Mode_shapes_circle[:,4]),coordinates,'justShow',5)

## Ibrahim Time Domain Method

Implement the Ibrahim Time Domain method to use it on the data from the impact hammer experiemnt.
Since the ITD operates on the free oscillation response in terms of the degrees of freedom (displacements in our case) you need to either compute the impulse respone from the accelerance using an inverse FFT or use the provided displacement data in `measurement-ITD.txt`.

* implement the ITD alogithm. Use the following skeleton

```python
def ITD(D,dt=1.0,n1=1,n3=17,tol=1.0):
    """Ibrahim time domain identification alogithm
    
    Parameters
    ----------
    D : array(N,M)
        matrix containing data at N measurement positions
        time sampled at M times
    dt : float
        sampling interval
    n1 : int
        number samples to shift between measurement matrix X1 and
        its time shifted companion X2
    n3 : int
        number samples to shift between upper and lower part of
        X1 and X2, respectively
    tol : float
        tolerance for the MSCCF used to sort out computational modes.
        only modes abs(MSCCF-1)<tol are returned. 
        
    Returns
    -------
    lam : array(K)
        identified eigenvalues, only positive frequencies are returned
    V : array(N,K)
        corresponding eigenvectors
    MSCCF : array(K)
        the mode shape correlation and confidence factor
    """
    
    # 1) determine which samples should be selected in X1 and X2
    Nt = D.shape[1]
    It1 = arange(0,Nt-n3-n1,1)
    # 2) select samples from data matrix
    X1u = D[:,It1] # upper part
    X1l = D[:,It1+n3] # lower part (shifted by n3 samples)
    X2u = D[:,It1+n1] # time shifted companion (shifted by n1 samples)
    X2l = D[:,It1+n1+n3]
    # 3) assmble X1 and X2
    # 4) compute A: use a robust inverse like numpy's linalg.pinv, or solve linear systems
    # 5) solve eigenvalue problem
    # 6) convert eigenvalues
    # 7) compute MSCCF and choose what to return
    
    return lam, V
```

* use the ITD to identify modes from the impulse response. Choose what data to take (which time range is useful?), and what sampling parameters are suitable. Plot the input data over time (all signals in one plot).

* plot the obtained modes and give their natural frequencies, damping factors, and MSCCFs.

## Compare mode shapes

Use the MAC / MSF to compare your identified modes with the deformation modes of the free-free plate.

* Compute the free-free modes and interpolate/evaluate them at the measurement locations.

* Compute the MAC-matrix between the "experimental" modes from peak-picking and the "numeric" free-free modes. Plot the matrix and ude it to identify the best-fitting "experimental" mode for each "numeric" deformation mode.

* Plot the mode shapes of the correlated mode pair to check. Do the frequencies match? Which threshold value of the MAC is suitable to select "correlated" modes?

* Do the same for the modes from circle fitting. Mark the impact location in the plots. Give frequencies and damping ratios, as well as absolute value and phase of the MSF between "numeric" and "experimental" modes.

* Do the same for the modes form the ITD